In [15]:
# Import the dependencies.
import pandas as pd
import matplotlib.pyplot as plt
import numpy as np
from citipy import citipy


In [16]:
# Create a set of random latitude and longitude combinations.

lats = np.random.uniform(low=-90.00,high=90.00,size=1500)
lngs = np.random.uniform(low=-180.00,high=180.00,size=1500)

lat_lngs = zip(lats,lngs)

lat_lngs

In [17]:
coordinates =list(lat_lngs)

In [18]:
cities =[]

for coordinate in coordinates:
    
    city = citipy.nearest_city(coordinate[0],coordinate[1]).city_name

    if city not in cities:

        cities.append(city)

len(cities)

602